# Extraction des heures à partir des fichiers textes dans Dropbox

Mon script de suivi des heures entre les nouvelles entrées comme des fichiers textes sur Dropbox. Je veux les synthétiser en un seul tableur, avec les informations pertinentes, pour faciliter l'entrée dans la feuille de temps ensuite.

In [1]:
import pandas as pd
from pathlib import Path

Les fichiers sont stockés dans un sous-dossier Dropbox quand tout va bien, et parfois par erreur dans le dossier racine. On pourra s'occuper des fichiers dans le sous-dossier pour commencer.

In [7]:
racine = Path.home() / 'Dropbox/'
boite_de_dépôt = racine / 'Boite de dépôt'

fichiers_correctement_déposés = list(boite_de_dépôt.glob('*.txt'))
fichiers_incorrectement_déposés = list(racine.glob('*.txt'))

Un fichier typique ressemble à ceci:

In [8]:
with fichiers_correctement_déposés[0].open() as f:
    print(f.read())

Temps (min): 45
Tâche: "Nouveau panneau pour Étienne "
Atelier: 1


On peut les lire ligne par ligne pour produire une liste:

In [16]:
with fichiers_correctement_déposés[0].open() as f:
    nouvelle_entrée = [0, '', False]
    
    for ligne in f.readlines():
        champ, valeur = ligne.strip().split(':')
        if champ.startswith('Temps'):
            # Certains fichiers ont le temps en heures, d'autres en minutes
            champ = champ.strip(')').split()
            if len(champ) > 1:
                unité = champ[1].strip('(')
            else:
                unité = 'min'
            
            valeur = float(valeur.strip())
            if unité == 'h':
                valeur *= 60
            
            nouvelle_entrée[0] = valeur
        elif champ.startswith('Tâche'):
            valeur = valeur.strip(' "')
            nouvelle_entrée[1] = valeur
        elif champ.startswith('Atelier'):
            nouvelle_entrée[2] = '1' in valeur

print(nouvelle_entrée)

[45.0, 'Nouveau panneau pour Étienne', True]


On utilise la date & heure de création du fichier (heureusement préservée par Dropbox) pour obtenir le moment d'entrée.

In [13]:
étampe = fichiers_correctement_déposés[0].stat().st_birthtime

In [14]:
from datetime import datetime as Dt
Dt.fromtimestamp(étampe)

datetime.datetime(2020, 12, 7, 16, 19, 8)

On peut l'ajouter à la liste:

In [17]:
with fichiers_correctement_déposés[0].open() as f:
    nouvelle_entrée = [None, 0, '', False]
    
    étampe = Dt.fromtimestamp(fichiers_correctement_déposés[0].stat().st_birthtime)
    nouvelle_entrée[0] = étampe
    
    for ligne in f.readlines():
        champ, valeur = ligne.strip().split(':')
        if champ.startswith('Temps'):
            # Certains fichiers ont le temps en heures, d'autres en minutes
            champ = champ.strip(')').split()
            if len(champ) > 1:
                unité = champ[1].strip('(')
            else:
                unité = 'min'
            
            valeur = float(valeur.strip())
            if unité == 'h':
                valeur *= 60
            
            nouvelle_entrée[1] = valeur
        elif champ.startswith('Tâche'):
            valeur = valeur.strip(' "')
            nouvelle_entrée[2] = valeur
        elif champ.startswith('Atelier'):
            nouvelle_entrée[3] = '1' in valeur

print(nouvelle_entrée)

[datetime.datetime(2020, 12, 7, 16, 19, 8), 45.0, 'Nouveau panneau pour Étienne', True]


Pour que ce soit vraiment utile, il faut passer à travers toute la liste de fichiers, et garder chaque entrée dans une liste.

In [23]:
entrées = []

for chemin in fichiers_correctement_déposés:
    nouvelle_entrée = [None, 0, '', False]
    
    with chemin.open() as f:
        étampe = Dt.fromtimestamp(chemin.stat().st_birthtime)
        nouvelle_entrée[0] = étampe

        for ligne in f.readlines():
            champ, valeur = ligne.strip().split(':', 1)
            if champ.startswith('Temps'):
                # Certains fichiers ont le temps en heures, d'autres en minutes
                champ = champ.strip(')').split()
                if len(champ) > 1:
                    unité = champ[1].strip('(')
                else:
                    unité = 'min'

                valeur = float(valeur.replace(',', '.').strip())
                if unité == 'h':
                    valeur *= 60

                nouvelle_entrée[1] = valeur
            elif champ.startswith('Tâche'):
                valeur = valeur.strip(' "')
                nouvelle_entrée[2] = valeur
            elif champ.startswith('Atelier'):
                nouvelle_entrée[3] = '1' in valeur
    
    entrées.append(nouvelle_entrée)

Pour faciliter la manipulation des données, on les transforme en cadre Pandas:

In [27]:
données = pd.DataFrame(entrées, columns=['Date', 'Temps (min)', 'Description', 'Atelier'])

In [28]:
données.to_excel(str(boite_de_dépôt / 'résumé.xls'))

On répète pour les fichiers qui n'étaient pas à la bonne place:

In [30]:
entrées = []

for chemin in fichiers_incorrectement_déposés:
    nouvelle_entrée = [None, 0, '', False]
    
    with chemin.open() as f:
        étampe = Dt.fromtimestamp(chemin.stat().st_birthtime)
        nouvelle_entrée[0] = étampe

        for ligne in f.readlines():
            champ, valeur = ligne.strip().split(':', 1)
            if champ.startswith('Temps'):
                # Certains fichiers ont le temps en heures, d'autres en minutes
                champ = champ.strip(')').split()
                if len(champ) > 1:
                    unité = champ[1].strip('(')
                else:
                    unité = 'min'

                valeur = float(valeur.replace(',', '.').strip())
                if unité == 'h':
                    valeur *= 60

                nouvelle_entrée[1] = valeur
            elif champ.startswith('Tâche'):
                valeur = valeur.strip(' "')
                nouvelle_entrée[2] = valeur
            elif champ.startswith('Atelier'):
                nouvelle_entrée[3] = '1' in valeur
    
    entrées.append(nouvelle_entrée)

données = pd.DataFrame(entrées, columns=['Date', 'Temps (min)', 'Description', 'Atelier'])
données.to_excel(str(boite_de_dépôt / 'résumé2.xls'))